Perform K-Means Clustering by selecting the optimal value of k and using Euclidean
distance as the similarity measure. Evaluate the algorithm under the following three
conditions, and observe the differences in the resulting clusters.
i. Maximum number of iterations
ii. Cluster centroid remains unchanged
iii. Highest quality of cluster is reached.

In [6]:
import numpy as np
import pandas as pd

data = pd.read_csv('data.csv')

def standardize(X):
    mean = np.mean(X, axis=0)
    std = np.std(X, axis=0)
    return (X - mean) / std

X = standardize(data.values)

def silhouette_score(X, labels):
    n = X.shape[0]
    silhouette_coefficients = []
    
    for i in range(n):
        cluster_i = labels[i]
        a_i = 0.00001
        b_i = float('inf')
        
        for j in range(n):
            if labels[j] == cluster_i:
                a_i += np.linalg.norm(X[i] - X[j])
        a_i /= len(X[labels == cluster_i]) - 1
        
        for c in set(labels):
            if c != cluster_i:
                b_i_c = 0
                for j in range(n):
                    if labels[j] == c:
                        b_i_c += np.linalg.norm(X[i] - X[j])
                b_i_c /= len(X[labels == c])
                b_i = min(b_i, b_i_c)
        
        silhouette_coefficients.append((b_i - a_i) / max(a_i, b_i))
    
    return np.mean(silhouette_coefficients)

def k_means(X, k, max_iter=100, tol=1e-4):
    centroids = X[np.random.choice(X.shape[0], k, replace=False)]
    
    for _ in range(max_iter):
        distances = np.sqrt(((X[:, None] - centroids) ** 2).sum(-1))
        labels = np.argmin(distances, axis=1)
        
        new_centroids = np.array([X[labels == i].mean(axis=0) for i in range(k)])
        
        if np.max(np.abs(new_centroids - centroids)) < tol:
            break
        centroids = new_centroids
    
    return labels, centroids

labels, centroids = k_means(X, k=3, max_iter=100)
print("Silhouette Score (Max Iterations):", silhouette_score(X, labels))
print("Labels (Max Iterations):", labels)

labels, centroids = k_means(X, k=3, max_iter=300, tol=1e-6)
print("Silhouette Score (Centroid Unchanged):", silhouette_score(X, labels))
print("Labels (Centroid Unchanged):", labels)

labels, centroids = k_means(X, k=3, max_iter=300, tol=1e-6)
print("Silhouette Score (Highest Quality):", silhouette_score(X, labels))
print("Labels (Highest Quality):", labels)


Silhouette Score (Max Iterations): 0.25479840239436363
Labels (Max Iterations): [2 0 0 0 2 1 2 2 0 0 2 2 0 0 2 1 1 2 1 2]
Silhouette Score (Centroid Unchanged): 0.25479840239436363
Labels (Centroid Unchanged): [2 0 0 0 2 1 2 2 0 0 2 2 0 0 2 1 1 2 1 2]
Silhouette Score (Highest Quality): 0.2598034364359002
Labels (Highest Quality): [2 1 1 1 2 0 2 2 1 1 2 1 1 1 2 0 2 2 0 2]


Repeat the above question with K-medoids algorithm. Note the difference between
the final clusters.

In [7]:
import numpy as np
import pandas as pd

data = pd.read_csv('data.csv')

def standardize(X):
    mean = np.mean(X, axis=0)
    std = np.std(X, axis=0)
    return (X - mean) / std

X = standardize(data.values)

def silhouette_score(X, labels):
    n = X.shape[0]
    silhouette_coefficients = []
    
    for i in range(n):
        cluster_i = labels[i]
        a_i = 0.00001
        b_i = float('inf')
        
        for j in range(n):
            if labels[j] == cluster_i:
                a_i += np.linalg.norm(X[i] - X[j])
        a_i /= len(X[labels == cluster_i]) - 1
        
        for c in set(labels):
            if c != cluster_i:
                b_i_c = 0
                for j in range(n):
                    if labels[j] == c:
                        b_i_c += np.linalg.norm(X[i] - X[j])
                b_i_c /= len(X[labels == c])
                b_i = min(b_i, b_i_c)
        
        silhouette_coefficients.append((b_i - a_i) / max(a_i, b_i))
    
    return np.mean(silhouette_coefficients)

def k_means(X, k, max_iter=100, tol=1e-4):
    centroids = X[np.random.choice(X.shape[0], k, replace=False)]
    
    for _ in range(max_iter):
        distances = np.sqrt(((X[:, None] - centroids) ** 2).sum(-1))
        labels = np.argmin(distances, axis=1)
        
        new_centroids = np.array([X[labels == i].mean(axis=0) for i in range(k)])
        
        if np.max(np.abs(new_centroids - centroids)) < tol:
            break
        centroids = new_centroids
    
    return labels, centroids

def k_medoids(X, k, max_iter=100):
    medoids = X[np.random.choice(X.shape[0], k, replace=False)]
    
    for _ in range(max_iter):
        distances = np.zeros((X.shape[0], k))
        for i in range(k):
            distances[:, i] = np.linalg.norm(X - medoids[i], axis=1)
        
        labels = np.argmin(distances, axis=1)
        
        new_medoids = np.array([X[labels == i][np.argmin(np.sum(distances[labels == i][:, i], axis=0))] for i in range(k)])
        
        if np.array_equal(new_medoids, medoids):
            break
        medoids = new_medoids
    
    return labels, medoids

labels_kmeans, centroids = k_means(X, k=3, max_iter=100)
print("K-means Silhouette Score:", silhouette_score(X, labels_kmeans))

labels_kmedoids, medoids = k_medoids(X, k=3, max_iter=100)
print("K-medoids Silhouette Score:", silhouette_score(X, labels_kmedoids))

print("K-means Labels:", labels_kmeans)
print("K-medoids Labels:", labels_kmedoids)


K-means Silhouette Score: 0.1415776187847221
K-medoids Silhouette Score: 0.11755681868562531
K-means Labels: [1 2 2 1 1 0 2 1 2 1 0 1 2 2 0 0 0 0 0 0]
K-medoids Labels: [0 2 2 1 0 0 1 0 1 1 0 1 2 2 0 0 0 0 0 0]
